In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

from keras.layers  import Dense
from tensorflow.keras import layers

In [ ]:
NUM_TRAIN = 50_000  # there are 50000 training examples in CIFAR-10
NUM_TEST = 10_000  # there are 10000 test examples in CIFAR-10
NUM_NRF = 6  # number of radial basis functions
NUM_CLASSES = 10

INPUT_SIGNALS = {
    'cct': {'train': {'min': -2.9318686, 'max': 5.517675},
            'test': {'min': -2.8747954, 'max': 5.2203484},
            'signals': {'min': -2.9318686, 'max': 5.517675}},
    'eat': {'train': {'min': 4.619322e-05, 'max': 0.99732083},
            'test': {'min': 0.00015154292, 'max': 0.9963026},
            'signals': {'min': 4.619322e-05, 'max': 0.99732083}},
    'fnet': {'train': {'min': -12.315351, 'max': 16.69544}, 
             'test': {'min': -12.172157, 'max': 15.076705}, 
             'signals': {'min': -12.315351, 'max': 16.69544}},
    'conv': {'train': {'min': 0.0, 'max': 1.0}, 
             'test': {'min': 0.0, 'max': 1.0}, 
             'signals': {'min': 0.0, 'max': 1.0}},
    'gmlp': {'train': {'min': -10.501355, 'max': 25.318792}, 
             'test': {'min': -10.048989, 'max': 22.385761}, 
             'signals': {'min': -10.501355, 'max': 25.318792}},
    'mlp_mixer': {'train': {'min': -14.840056, 'max': 26.011475},
                  'test': {'min': -14.86421, 'max': 24.304762},
                  'signals': {'min': -14.86421, 'max': 26.011475}},
    'swin_trans': {'train': {'min': 3.1436898e-07, 'max': 0.9996327}, 
                   'test': {'min': 3.2155782e-07, 'max': 0.99982965}, 
                   'signals': {'min': 3.1436898e-07, 'max': 0.99982965}},
}

In [ ]:
nn_types = ['cct', 'eat', 'fnet', 'conv', 'gmlp', 'mlp_mixer', 'swin_trans']  # specify NNs

for nn_type in nn_types:
    if nn_type not in INPUT_SIGNALS:
        raise ValueError(f"Unknown neural network type - {nn_type}.")

In [ ]:
def rf(x, mu, sigma):
    return np.exp(- ((x - mu) ** 2)/(2*(sigma**2)))


def prepare_data(nn_type: str):
    print(f"Preparing data for '{nn_type}' network")

    X_test = [np.zeros((NUM_TEST, NUM_NRF), 'float') for _ in range(NUM_CLASSES)]
    X_train = [np.zeros((NUM_TRAIN, NUM_NRF), 'float') for _ in range(NUM_CLASSES)]
    xvs = np.zeros((NUM_NRF), 'float')  # array of radial basis functions centers
    
    # base_vars
    x_min = INPUT_SIGNALS[nn_type.lower()]['signals']['min']
    x_max = INPUT_SIGNALS[nn_type.lower()]['signals']['max']
    work_dir = os.getcwd()
    train_file_path = os.path.join(work_dir, 'resources', 'input-data', nn_type.lower(), 's_pred_train.npy')
    test_file_path = os.path.join(work_dir, 'resources', 'input-data', nn_type.lower(), 's_pred_test.npy')
    
    xvs[0] = x_min
    xvs[NUM_NRF-1] = x_max
    d = (x_max - x_min) / (NUM_NRF-1)
    
    for i in range(NUM_NRF-2):
        xvs[i+1] = x_min + d*(i+1)
    for i in range(NUM_NRF):
        print(f"xvs[{i}] = {xvs[i]}")
    
    X = np.linspace(x_min, x_max, 1000)
    sigma=(x_max-x_min)/ 8.4495436
    for i in range(NUM_NRF):
        mu = xvs[i]
        plt.plot(X, rf(X, mu, sigma), color = 'b')
    plt.show()
    
    # Load input data values
    X_input_test = np.load(test_file_path)
    X_input_train = np.load(train_file_path)
    
    for i in range(NUM_TEST):
        for j in range(NUM_CLASSES):
            for k in range(NUM_NRF):
                X_test[j][i, k] = rf(X_input_test[i, j], xvs[k], sigma)

    for i in range(NUM_TRAIN):
        for j in range(NUM_CLASSES):
            for k in range(NUM_NRF):
                X_train[j][i, k] = rf(X_input_train[i, j], xvs[k], sigma)
                
    return X_train, X_test

In [ ]:
train_data = []
test_data = []

for nn_type in nn_types:
    prep_train_data, prep_test_data = prepare_data(nn_type)
    
    train_data.extend(prep_train_data)
    test_data.extend(prep_test_data)
    
train_data = {f"inp{i}": train_data[i] for i in range(len(train_data))}
test_data = {f"inp{i}": test_data[i] for i in range(len(test_data))}

In [ ]:
# Loading Cifar10 for y_train and y_test
(_, y_train), (_, y_test) = keras.datasets.cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

input_layers = [keras.Input(shape=(NUM_NRF,), name=f'inp{i}') for i in range(len(train_data))]

output_layers = [Dense(1, kernel_initializer='glorot_uniform', use_bias=None)(input_layers[i]) for i in range(len(train_data))]

x = layers.concatenate(output_layers)
out = Dense(NUM_CLASSES, activation='softmax')(x)
model = keras.Model(inputs=input_layers, outputs=out)
model.summary()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

In [ ]:
NUM_EPOCHS = 20
BATCH_SIZE = 32

history = model.fit(
    train_data, 
    y_train,
    epochs=NUM_EPOCHS, 
    batch_size=BATCH_SIZE,
    validation_data=(
        test_data, 
        y_test
    )
)

In [ ]:
plt.style.use("seaborn-v0_8")
f, ax = plt.subplots(1, 2, figsize=(15, 6))

ax[0].plot(history.history['val_loss'],label='Testing Loss')
ax[0].plot(history.history['loss'],label='Training Loss')
ax[0].set_title('Losses',weight='bold',size='x-large')
ax[0].set_xlabel('Epoch',size='large')
ax[0].set_ylabel('Loss',size='large')
ax[0].legend()

ax[1].plot(history.history['val_accuracy'],label='Testing Accuracy')
ax[1].plot(history.history['accuracy'],label='Training Accuracy')
ax[1].set_title('Accuracies',weight='bold',size='x-large')
ax[1].set_xlabel('Epoch',size='large')
ax[1].set_ylabel('Accuracy',size='large')
ax[1].legend()

plt.show()

In [ ]:
model.get_weights()